In [52]:
import pandas as pd
import csv
import numpy as np
import sys
import os
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import re
from statistics import mean
from num2words import num2words
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import math
from collections import Counter
from pprint import pprint
from copy import deepcopy
from sent2vec.vectorizer import Vectorizer
from scipy import spatial

np.set_printoptions(threshold=sys.maxsize)        #enables to see all the rows of the np array when printed which is otherwise concatenated


In [3]:
os.chdir("D:\_NLP_Sahaj\Data Generation\data_generation")
%run data_gen_mp.py                                                   #change the directory to the data_generation file and run the data generation file.

In [5]:
txt=open("D:\_NLP_Sahaj\Data Generation\data_generation\output1.txt").read()
sentences=txt.splitlines()                                             #using splitlines() function to split the text at each \n character.This works, because each thread of the conversation starts at a new line. Realised that this was the easiest and best way, after trying sentence segmentation using spacy and nltk.


In [6]:
print(sentences)  #print the sentences after splitting at each newline character

['=======START============', 'start', 'Hi, I am looking to buy 1000 paper plates, delivered at delhi', "Sorry, we don't have that many of those right now", 'Oh okay', 'Do you have plastic spoons?', 'Yeah,', 'How many pieces are you looking for?', 'I need 1000 pieces', "Sorry, we don't have that many of those right now", 'end', '=======END============', '=======START============', 'start', 'Hey Krishna stationary, Do you supply ruled books', 'How many pieces are you looking for?', 'I need 1000 pieces', 'Yes, we can supply', 'Do you also happen to have plain books?', 'Sorry, presently out of stock of plain books', "Oh okay, let's go ahead with ruled books then", 'Our price would be 40 per ruled book', 'The price seems a bit too high, thank you', 'end', '=======END============', '=======START============', 'start', 'Hi, I am looking to buy apples', 'What is the quantity you need', '300', "Sorry, we don't have that many of those right now", 'Oh,fine', ' I am also looking for oranges', 'Yes

In [7]:
print(is_buyer)      #The data_generation file generates this list: is_buyer.This is actually a list of lists. Each sub-list corresponding to each conversation.Each line in the conversation is given either 0/1. 0 if it is a seller dialogue and 1 if it is a buyer dialogue.

[[1, 0, 1, 1, 0, 0, 1, 0], [1, 0, 1, 0, 1, 0, 1, 0, 1], [1, 0, 1, 0, 1, 1, 0, 0, 1, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0, 0, 1, 0, 1, 0], [1, 0, 1, 0, 0, 1, 0, 0, 1, 0], [1, 0, 1, 0, 1], [1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1], [1, 0, 1, 0, 0, 1, 0], [1, 0, 0, 1, 0, 1, 0, 1], [1, 0, 1, 1, 0, 0, 1, 0], [1, 0, 1, 0, 0, 1, 1, 0, 1], [1, 0, 1], [1, 0, 1, 0, 1], [1, 0, 1, 0, 1, 0, 1], [1, 0], [1, 0], [1, 0, 1], [1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1], [1, 0, 1, 0, 1, 0, 1], [1, 0, 1, 1, 0], [1, 0, 1, 0, 1], [1, 0, 1, 0, 1], [1, 0], [1, 0, 1, 0, 1], [1, 0, 1, 0, 1, 0, 1], [1, 0, 1, 0, 0, 1, 0, 1, 0, 1], [1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0, 0, 1, 0], [1, 0], [1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1], [1, 0], [1, 0, 1, 1, 0, 1, 0, 1], [1, 0, 1, 1, 0, 0, 1, 0, 1, 0], [1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1], [1, 0], [1, 0, 1, 0, 1, 1, 0, 1], [1, 0], [1, 0, 1, 0, 1, 0, 1, 0], [1, 0, 1, 0], [1, 0], [1, 0, 1, 0, 0, 1, 1, 0, 1], [1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0], [1, 0, 1, 0, 1

In [8]:
#function to clean sentences
def clean_text(sen):
    # Remove punctuations
    sen = re.sub('[^a-zA-Z0-9]', ' ', sen)
    return sen.lower()

In [11]:
# removing the unneccesary start and end threads from the list of sentences.Only the 'start' threads are kept to distinguish between conversations.
brakers =['=======START============', '=======END============' , 'end']

for sentence in sentences:
    if sentence in brakers:
        sentences.remove(sentence)

In [12]:
#text-cleaning
for i,sentence in enumerate(sentences):
   sentences[i]= (clean_text(sentence))

In [13]:
#printing the sentences after removing of the 'brakers' and cleaning the text
print(sentences)

['start', 'hi  i am looking to buy 1000 paper plates  delivered at delhi', 'sorry  we don t have that many of those right now', 'oh okay', 'do you have plastic spoons ', 'yeah ', 'how many pieces are you looking for ', 'i need 1000 pieces', 'sorry  we don t have that many of those right now', 'start', 'hey krishna stationary  do you supply ruled books', 'how many pieces are you looking for ', 'i need 1000 pieces', 'yes  we can supply', 'do you also happen to have plain books ', 'sorry  presently out of stock of plain books', 'oh okay  let s go ahead with ruled books then', 'our price would be 40 per ruled book', 'the price seems a bit too high  thank you', 'start', 'hi  i am looking to buy apples', 'what is the quantity you need', '300', 'sorry  we don t have that many of those right now', 'oh fine', ' i am also looking for oranges', 'yes ', 'where is your location ', 'chennai', 'sorry  we can t deliver to that location', 'start', 'hey ramesh retail do you supply paper plates  i need 1

In [14]:
#parsing the list of sentences into conversations using the 'start' as a delimiter.All the conversations are stored in the list called conversations
conversations=[]
for sentence in sentences:
    if sentence =='start':
        conversations.append([])
        continue
    conversations[-1].append(sentence)

In [18]:
print(conversations[0],'\n\n',conversations[1],'\n\n',conversations[2],'\n\n')   #printing the first 3 conversations

['hi  i am looking to buy 1000 paper plates  delivered at delhi', 'sorry  we don t have that many of those right now', 'oh okay', 'do you have plastic spoons ', 'yeah ', 'how many pieces are you looking for ', 'i need 1000 pieces', 'sorry  we don t have that many of those right now'] 

 ['hey krishna stationary  do you supply ruled books', 'how many pieces are you looking for ', 'i need 1000 pieces', 'yes  we can supply', 'do you also happen to have plain books ', 'sorry  presently out of stock of plain books', 'oh okay  let s go ahead with ruled books then', 'our price would be 40 per ruled book', 'the price seems a bit too high  thank you'] 

 ['hi  i am looking to buy apples', 'what is the quantity you need', '300', 'sorry  we don t have that many of those right now', 'oh fine', ' i am also looking for oranges', 'yes ', 'where is your location ', 'chennai', 'sorry  we can t deliver to that location'] 




In [19]:
os.chdir("D:\_NLP_Sahaj\Data")    #changing the directory to where all the data like positive words.txt ,negative words.txt and other files are contained.

In [25]:
#Define a function to tokenize the given conversation
def tokenizer(doc):
    token_pattern = re.compile(r"(?u)\b\w\w+\b")
    return token_pattern.findall(doc)

In [20]:
#Function to make a dictionary of positive and negative words using the files containing the positive and negative words
def load_sentiment_lexicon(pos_file, neg_file):
    pos_dict, neg_dict = {}, {}

    with open(pos_file) as fin:
        for line in fin:
            if line != '\n':
                line = line.strip()
                pos_dict[line] = 1

    with open(neg_file) as fin:
        for line in fin:
            if line != '\n':
                line = line.strip()
                neg_dict[line] = 1

    return pos_dict, neg_dict

In [22]:
#Functions for sentiment feature extraction

#Function to extract sentiment scores of each line of a given conversation
def sentiment_scores(sentences):
    res = []
    sid = SentimentIntensityAnalyzer()
    for sentence in sentences:
        ss = sid.polarity_scores(sentence)
        res.append(list ([ss['neg'], ss['neu'], ss['pos']]))
    return res

#Function which tells whether the word 'thank' is there in each line of a given conversation
def thank(utterances):
    res = []
    for utterance in utterances:
        if utterance.find('thank') != -1:
            res.append(1)
        else:
            res.append(0)
    return res

#Function which tells whether there is an exclamation mark in each line of a given conversation
def exclamation_mark(utterances):
    res = []
    for utterance in utterances:
        if utterance.find('!') != -1:
            res.append(1)
        else:
            res.append(0)
    return res

#Function which tells whether there is the presence of 'does not' or 'did not' in each line of a given conversation
def ve_feedback(utterances):
    res = []
    for utterance in utterances:
        if utterance.find('did not') != -1 or utterance.find('does not') != -1:
            res.append(1)
        else:
            res.append(0)
    return res

#Function to extract number of positive and negative words in each line of a given conversation
def lexicon(utterances, pos_dict, neg_dict):
    res = []
    for utterance in utterances:
        pos_count, neg_count = 0, 0
        tokens = tokenizer(utterance)
        for token in tokens:
            if token in pos_dict:
                pos_count += 1
            elif token in neg_dict:
                neg_count += 1
        res.append([(pos_count), (neg_count)])
    return res




In [26]:
    #Just testing out the functionality of the positive and negaitve word counter function
    utterances = ['bad movie', 'thank you for your help, but the solution does not work']
    pos_file = 'positive-words.txt'
    neg_file = 'negative-words.txt'
    pos_dict, neg_dict = load_sentiment_lexicon(pos_file, neg_file)
    print(lexicon(utterances, pos_dict, neg_dict))

[[0, 1], [2, 0]]


In [28]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

#Function to extract number of tokens,unique number of tokens and unique number of tokens after stemming in each line of a given conversation
def post_length(utterances):
    length = []
    unique_length = []
    unique_stemmed_length = []

    stop = set(stopwords.words('english'))
    ps = PorterStemmer()

    for utterance in utterances:
        tokens = tokenizer(utterance)
        tokens = [token for token in tokens if token not in stop]
        unique_tokens = list(set(tokens))
        #print(unique_tokens)
        unique_tokens_stemmed = list(set([ps.stem(token) for token in unique_tokens]))

        length.append(len(tokens))
        unique_length.append(len(unique_tokens))
        unique_stemmed_length.append(len(unique_tokens_stemmed))

    return length, unique_length, unique_stemmed_length



In [30]:
#Testing the basic functionality of the above function
utterances = ['play, played, plays, played,player,plays', 'Hi!, my name is Nikhil. What is yours']
length, unique_length, unique_stemmed_length = post_length(utterances)
print(length)
print(unique_length)
print(unique_stemmed_length)

[6, 4]
[4, 4]
[2, 4]


In [31]:
#Function which tells whether there is a Question mark in each line of a given conversation
def question_mark(utterances):
    res = []
    for utterance in utterances:
        if utterance.find('?') != -1:
            res.append(1)
        else:
            res.append(0)
    return res

#Function which tells whether there is the presence of 'same' or'similar' in each line of a given conversation
def duplicate(utterances):
    res = []
    for utterance in utterances:
        if utterance.find('same') != -1 or utterance.find('similar') != -1:
            res.append(1)
        else:
            res.append(0)
    return res

#Function which tells whether there is the presence of how,what,why,who,where and when in each line of a given conversation
def W5H1(utterances):
    # how, what, why, who, where, when
    res = []
    for utterance in utterances:
        wh_vector = [0] * 6

        wh_vector[0] = 1 if utterance.find('how') != -1 else 0
        wh_vector[1] = 1 if utterance.find('what') != -1 else 0
        wh_vector[2] = 1 if utterance.find('why') != -1 else 0
        wh_vector[3] = 1 if utterance.find('who') != -1 else 0
        wh_vector[4] = 1 if utterance.find('where') != -1 else 0
        wh_vector[5] = 1 if utterance.find('when') != -1 else 0


        res.append(list( wh_vector))
    return res



In [33]:
#Testing the functionality of the above function
utterances = ['Hi!, my name is Nikhil. what is yours?', 'who are you']
print(W5H1(utterances))
 

[[0, 1, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0]]


In [35]:
#Function which upon giving a conversation returns a list of the tf-idf cosine similarity of each sentence in the conversation wrt to the initial sentence and also a list of the similarity of each conversation wrt to the whole conversation

def cosine_similarity(utterances, term_to_idf_dict):
    init_sim, thread_sim = [], []
    
    utterances_tokens,thread_tokens = [],[]

    for i, utterance in enumerate(utterances):
        utterance_tokens = list(filter(lambda x: x in term_to_idf_dict, tokenizer(utterance)))
        utterances_tokens.append(utterance_tokens)

    # pprint(utterances_tokens)

    init_post = deepcopy(utterances_tokens[0])
    for utterance_tokens in utterances_tokens:
        thread_tokens += utterance_tokens

    for utterance_tokens in utterances_tokens:
        init_sim.append(compute_tf_idf_cosine(init_post, utterance_tokens, term_to_idf_dict))
    for utterance_tokens in utterances_tokens:
        thread_sim.append(compute_tf_idf_cosine(thread_tokens, utterance_tokens, term_to_idf_dict))

    return  init_sim, thread_sim

def compute_tf_idf_cosine(q1, q2, term_to_idf_dict):
    q1_term_count = Counter(q1)
    q2_term_count = Counter(q2)
    idf = 10.8983491402  # default value
    for t in q1_term_count.keys():
        idf = term_to_idf_dict[t]
        q1_term_count[t] *= idf

    for t in q2_term_count.keys():
    
        idf = term_to_idf_dict[t]
        q2_term_count[t] *= idf

    cosine = 0.0
    q1_l2_norm = 0.0
    q2_l2_norm = 0.0
    for q1_term in q1_term_count.keys():
        q1_l2_norm += math.pow(q1_term_count[q1_term],2)
        if q1_term not in q2_term_count.keys():
            continue
        else:
            cosine += q1_term_count[q1_term] * q2_term_count[q1_term]

    for q2_term in q2_term_count.keys():
        q2_l2_norm += math.pow(q2_term_count[q2_term],2)
    #normalize
    # print('q1, q2: ', q1, q2)
    try:
        return cosine / (math.sqrt(q1_l2_norm) * math.sqrt(q2_l2_norm))
    except:
        return 0


In [37]:
#define a function to initialise the tf-idf dictionary using the downloaded "idf.tsv" file
def init_tf_idf_dict(idf_file):
    term_to_idf_dict = {}

    with open(idf_file, encoding='utf-8') as fin:
        for line in fin:
            tokens =  re.split('[-\s,\n]+', line)
            term_to_idf_dict[tokens[0]] = float(tokens[1])

    return term_to_idf_dict

term_to_idf_dict = init_tf_idf_dict("idf.tsv")

In [44]:
#Testing the cosine similarity function
for i,conversation in enumerate(conversations):
           if i>20: 
            break
           init_sim, thread_sim = cosine_similarity(conversation, term_to_idf_dict)
           print(conversation)
           print(init_sim,thread_sim,'\n')
             

['hi  i am looking to buy 1000 paper plates  delivered at delhi', 'sorry  we don t have that many of those right now', 'oh okay', 'do you have plastic spoons ', 'yeah ', 'how many pieces are you looking for ', 'i need 1000 pieces', 'sorry  we don t have that many of those right now']
[0.9999999999999998, 0.0, 0.0, 0.0, 0.0, 0.1132106566092044, 0.0, 0.0] [0.5246638787186257, 0.6151500764804522, 0.2715314850787437, 0.250808419042548, 0.22027290950319348, 0.5127107821645244, 0.36846936431804606, 0.6151500764804522] 

['hey krishna stationary  do you supply ruled books', 'how many pieces are you looking for ', 'i need 1000 pieces', 'yes  we can supply', 'do you also happen to have plain books ', 'sorry  presently out of stock of plain books', 'oh okay  let s go ahead with ruled books then', 'our price would be 40 per ruled book', 'the price seems a bit too high  thank you']
[1.0, 0.013564859764432627, 0.0, 0.1878122910922464, 0.13267453249994252, 0.07064605139707718, 0.2859210932643148, 0.

In [49]:
#Alternate function for cosine similarity(which is not being used right now) with Google sentence2vec. However this is only for finding similarity between two sentences and not sure how to use this to find similarity btw sentence and whole conversation.This also takes really lot of time to run 

def bert_cosine_similarity(conversation):
 
        vectorizer =Vectorizer()
        vectorizer.bert([conversation[i] for i in range(len(conversation))])
        vectors=vectorizer.vectors
        dist_first=[spatial.distance.cosine(vectors[0],vectors[i]) for i in range(len(conversation))]
       
        '''dist_total=[]
        for i in range(len(conversation)):
            dist=[spatial.distance.cosine(vectors[i],vectors[j]) for j in range(len(conversation))]
            dist_total.append(mean(dist))'''
        return(dist_first)
        

In [53]:
#testing the above function
for i,conversation in enumerate(conversations):
    if i ==8:
        a=bert_cosine_similarity(conversation)
        print(conversation,a)

['hey krishna stationary do you supply whiteboards  i need 60 whiteboards', 'where is your location ', 'kolkata', 'sure  i can supply ', 'it will cost 600 per whiteboard', 'can we make it 500 per  whiteboard ', 'your offer is difficult to work with  but lets go ahead'] [0.0, 0.07831662893295288, 0.10764431953430176, 0.07678711414337158, 0.06871199607849121, 0.05109453201293945, 0.06529533863067627]


In [47]:
#Generating the test features by running the feature extraction functions written on the conversations.One row of the feature vector is extracted from each line of each of the conversations and are subsequently staccked intp the np array:Test_features

test_features=np.zeros(24)   #24 is the number of features extracted from each line in the conversation
lines=[]
for n,conversation in enumerate(conversations):
    if n<20:        #only the first 20 conversations are taken
          
                init_sim, thread_sim = cosine_similarity(conversation, term_to_idf_dict)
                qm = question_mark(conversation)
                dup = duplicate(conversation)
                wh = W5H1(conversation)

                # structural features
                abs_pos = [idx + 1 for idx in range(len(conversation))]
                norm_pos = [pos / len(conversation) for pos in abs_pos]
                length, unique_length, unique_stemmed_length = post_length(conversation)
                is_starter = is_buyer[n]
            

                # sentiment based features
                thx = thank(conversation)
                exclam_mark = exclamation_mark(conversation)
                vf = ve_feedback(conversation)
                ss = sentiment_scores(conversation)
                lexicon_counts = lexicon(conversation, pos_dict, neg_dict)
                for sentence in conversation:
                    lines.append(sentence)  
                for i in range(len(conversation)):
                    
                    test_features=np.vstack((test_features,np.array([init_sim[i],thread_sim[i],qm[i],dup[i],wh[i][0],wh[i][1],wh[i][2],wh[i][3],wh[i][4],wh[i][5],abs_pos[i],norm_pos[i],length[i],unique_length[i],unique_stemmed_length[i],is_starter[i],thx[i],exclam_mark[i],vf[i],ss[i][0],ss[i][1],ss[i][2],lexicon_counts[i][0],lexicon_counts[i][1]])))

test_features = np.delete(test_features, (0), axis=0)   #The first row is deleted beacuse the array was initiated with a row of zeros as seen in the first line
print(test_features)
                    
                
            

[[1.00000000e+00 5.24663879e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 1.00000000e+00 1.25000000e-01
  8.00000000e+00 8.00000000e+00 8.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 6.15150076e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 2.00000000e+00 2.50000000e-01
  3.00000000e+00 3.00000000e+00 3.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.26000000e-01
  8.74000000e-01 0.00000000e+00 1.00000000e+00 1.00000000e+00]
 [0.00000000e+00 2.71531485e-01 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 3.00000000e+00 3.75000000e-01
  2.00000000e+00 2.00000000e+00 2.00000000e+00 1.00000000e+00
  0.00

In [54]:
#Random forest class prediction



#Building feature vectors from the train_features.tsv file available in the MS Dialogue dataset.


os.chdir("D:\_NLP_Sahaj\Data\MSDialogue")       #changing the path to where the feature vectors for training are present["train_features.tsv"]
training_features_path = "train_features.tsv"     
df = pd.read_csv(training_features_path,sep='\t')


#Reading the "train_features.tsv and building the np array:xtrain which has the features and ytrain which has the labels for training"
x_train = np.zeros(24)
y_train = []
for i, row in enumerate(df.iterrows()):
    label = tuple((row[1][0]).split("_"))
    features = np.array((row[1][1]).split(" "), dtype='float64')
    x_train = np.vstack((x_train, features))
    y_train.append(label)
        
x_train = x_train[1:][:]

In [55]:
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(y_train)
print(list(mlb.classes_))

['CQ', 'FD', 'FQ', 'GG', 'IR', 'JK', 'NF', 'O', 'OQ', 'PA', 'PF', 'RQ']


In [56]:
#training using random forest

forest = RandomForestClassifier(n_estimators=100, random_state=1)
RandomForest = MultiOutputClassifier(forest, n_jobs=-1)
RandomForest.fit(x_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(random_state=1),
                      n_jobs=-1)

In [57]:
x_test_pred = RandomForest.predict(test_features)
x_test_pred=mlb.inverse_transform(x_test_pred)

In [58]:
y_test = mlb.transform(y_train)
print(list(mlb.classes_))

['CQ', 'FD', 'FQ', 'GG', 'IR', 'JK', 'NF', 'O', 'OQ', 'PA', 'PF', 'RQ']


D:\Anacondaa\lib\site-packages\sklearn\preprocessing\_label.py:860: UserWarning: unknown class(es) [0, 1] will be ignored
  warnings.warn('unknown class(es) {0} will be ignored'


In [62]:
for i in range(118):
    print(lines[i],x_test_pred[i],'\n',test_features[i],'\n')

hi  i am looking to buy 1000 paper plates  delivered at delhi ('OQ',) 
 [1.         0.52466388 0.         0.         0.         0.
 0.         0.         0.         0.         1.         0.125
 8.         8.         8.         1.         0.         0.
 0.         0.         1.         0.         0.         0.        ] 

sorry  we don t have that many of those right now () 
 [0.         0.61515008 0.         0.         0.         0.
 0.         0.         0.         0.         2.         0.25
 3.         3.         3.         0.         0.         0.
 0.         0.126      0.874      0.         1.         1.        ] 

oh okay () 
 [0.         0.27153149 0.         0.         0.         0.
 0.         0.         0.         0.         3.         0.375
 2.         2.         2.         1.         0.         0.
 0.         0.         0.345      0.655      0.         0.        ] 

do you have plastic spoons  ('FD',) 
 [0.         0.25080842 0.         0.         0.         0.
 0.         0.

In [ ]:
#code to try and change threshold of prediction to custom value
'''
threshold=0.1
x_test_pred = RandomForest.predict(test_features)
predicted = (x_test_pred >= threshold).astype('int')
x_test_pred=mlb.inverse_transform(predicted)
'''